In [3]:
from datasets import Dataset, load_dataset

In [4]:
dataset = load_dataset("text", data_files={"train": ["alluneedisattention.txt"]})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/jerry/.cache/huggingface/datasets/text/default-3b1d417f369cc092/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18
    })
})

In [7]:
train = dataset['train']

In [28]:
context = '''The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.'''
context = context.replace('\n', ' ')

In [29]:
context

'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English\x02to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Tra

In [30]:
l = []
for i in range(0, len(train), 2):
    q = train[i]['text']
    a = train[i+1]['text']
    q = q.split(':')[-1]
    a = a.split(':')[-1]
    l.append({'context':context, 'question':q, 'answer':a})

In [31]:
l

[{'context': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English\x02to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show

In [34]:
dataset = Dataset.from_list(l)

In [35]:
dataset

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 9
})

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [20]:
t_dataset = dataset.map(lambda examples: tokenizer(examples["question"]), batched=True)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [23]:
t_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9
})

In [22]:
t_dataset[0]

{'question': 'What is the title of the paper, and who are the authors?',
 'answer': 'The title of the paper is "Attention Is All You Need". The authors are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser and Illia Polosukhin.',
 'input_ids': [101,
  1327,
  1110,
  1103,
  1641,
  1104,
  1103,
  2526,
  117,
  1105,
  1150,
  1132,
  1103,
  5752,
  136,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
max_length = 512
doc_stride = 256

In [36]:
example = dataset[0]

In [37]:
example

{'context': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English\x02to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show 

In [38]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [39]:
tokenized_example

{'input_ids': [[101, 1327, 1110, 1103, 1641, 1104, 1103, 2526, 117, 1105, 1150, 1132, 1103, 5752, 136, 102, 1109, 7065, 4954, 14715, 11243, 3584, 1132, 1359, 1113, 2703, 1231, 21754, 1137, 14255, 6005, 18404, 1348, 18250, 6379, 1115, 1511, 1126, 4035, 13775, 1197, 1105, 170, 1260, 13775, 1197, 119, 1109, 1436, 4072, 3584, 1145, 7543, 1103, 4035, 13775, 1197, 1105, 1260, 13775, 1197, 1194, 1126, 2209, 6978, 119, 1284, 17794, 170, 1207, 3014, 2443, 4220, 117, 1103, 13809, 23763, 117, 1359, 9308, 1113, 2209, 10748, 117, 4267, 20080, 5026, 1158, 1114, 1231, 10182, 21629, 1105, 14255, 6005, 18404, 1116, 3665, 119, 28009, 1116, 1113, 1160, 3395, 5179, 8249, 1437, 1292, 3584, 1106, 1129, 7298, 1107, 3068, 1229, 1217, 1167, 5504, 23228, 2165, 1105, 8753, 5409, 1750, 1159, 1106, 2669, 119, 3458, 2235, 5515, 1116, 1743, 119, 125, 139, 17516, 2591, 1113, 1103, 160, 13910, 1387, 1483, 2430, 118, 1528, 5179, 4579, 117, 9248, 1166, 1103, 3685, 1436, 2686, 117, 1259, 24957, 117, 1118, 1166, 123, 139,

In [41]:
for x in tokenized_example["input_ids"]:
    print(tokenizer.decode(x))

[CLS] What is the title of the paper, and who are the authors? [SEP] The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28. 4 BLEU on the WMT 2014 Englishto - German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English - to - French translation task, our model establishes a new single - model state - of - the - art BLEU score of 41. 8 after training for 3. 5 days on eight GPUs, a small fraction o

In [42]:
tokenized_example2 = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)

In [43]:
tokenized_example2

{'input_ids': [[101, 1327, 1110, 1103, 1641, 1104, 1103, 2526, 117, 1105, 1150, 1132, 1103, 5752, 136, 102, 1109, 7065, 4954, 14715, 11243, 3584, 1132, 1359, 1113, 2703, 1231, 21754, 1137, 14255, 6005, 18404, 1348, 18250, 6379, 1115, 1511, 1126, 4035, 13775, 1197, 1105, 170, 1260, 13775, 1197, 119, 1109, 1436, 4072, 3584, 1145, 7543, 1103, 4035, 13775, 1197, 1105, 1260, 13775, 1197, 1194, 1126, 2209, 6978, 119, 1284, 17794, 170, 1207, 3014, 2443, 4220, 117, 1103, 13809, 23763, 117, 1359, 9308, 1113, 2209, 10748, 117, 4267, 20080, 5026, 1158, 1114, 1231, 10182, 21629, 1105, 14255, 6005, 18404, 1116, 3665, 119, 28009, 1116, 1113, 1160, 3395, 5179, 8249, 1437, 1292, 3584, 1106, 1129, 7298, 1107, 3068, 1229, 1217, 1167, 5504, 23228, 2165, 1105, 8753, 5409, 1750, 1159, 1106, 2669, 119, 3458, 2235, 5515, 1116, 1743, 119, 125, 139, 17516, 2591, 1113, 1103, 160, 13910, 1387, 1483, 2430, 118, 1528, 5179, 4579, 117, 9248, 1166, 1103, 3685, 1436, 2686, 117, 1259, 24957, 117, 1118, 1166, 123, 139,

In [44]:
offsets = tokenized_example2['offset_mapping']

In [47]:
len(offsets[0])

260

In [48]:
len(tokenized_example2['input_ids'][0])

260

In [ ]:
max_input_length = 512
# max_target_length = 128

def preprocess_function(examples):
    # inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(examples['question'], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["answer"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs